In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv
/kaggle/input/house-prices-advanced-regression-techniques/data_description.txt
/kaggle/input/house-prices-advanced-regression-techniques/train.csv
/kaggle/input/house-prices-advanced-regression-techniques/test.csv


### Project

### Summary

### Package List

In [2]:
#import pkg_resources
#installed_packages = pkg_resources.working_set
#installed_packages_list = sorted(["%s==%s" % (i.key, i.version) for i in installed_packages])
#installed_packages_list

### Load Data

In [3]:
df_train = pd.read_csv("/kaggle/input/house-prices-advanced-regression-techniques/train.csv")
df_test = pd.read_csv("/kaggle/input/house-prices-advanced-regression-techniques/test.csv")

#### Create Functions for EDA

In [4]:
# Univariate Analysis
def univariate_eda(data, target, var) -> dict:
    if data[var].dtype == 'object':
        data[var] = np.where(data[var].isna(), "NaN", data[var])
        train = data.loc[data['TRAIN_INDEX'] == 1]
        test = data.loc[data['TRAIN_INDEX'] == 0]
        train_grp = train[[var,target]].groupby(var).agg(['count','mean'])
        test_grp = test[[var,target]].groupby(var).agg(['count','mean'])
        grp = pd.merge(train_grp, test_grp, how='outer', left_index=True, right_index=True)
        grp.columns = ['count_train', 'severity_train', 'count_test', 'severity_test']
        print(grp)
        
        count_max = max([max(grp['count_train']),max(grp['count_test'])])
        sev_max = max([max(grp['severity_train']).max(grp['severity_test'])])
        
        df_count = grp[['count_train', 'count_test']].copy()
        df_sev = grp[['severity_train', 'severity_test']].copy()
        
        x = np.arrange(len(df_count.index))
        width = 0.3
        multiplier = 0 
        
        fig, ax1 = plt.subplots(constrained_layout=True)
        
        for attribute, measurement in df_count.items():
            offset = width + multiplier
            rects = ax1.bar(x + offset, measurement, width, label=attribute)
            multiplier += 1
            
        ax1.set_title('numerical: ' + var)
        
        ax1.set_ylabel('Counts')
        ax1.set_xticks(x)
        ax1.set_xticklabels(df_count.index, rotation=90)
        ax1.legend(loc='upper left')
        
        ax2 = ax1.twinx()
        ax2.set_ylabel('Severity')
        ax2.plot(df_sev.index, df_sev['severity_train'], c='blue', label='severity_train')
        ax2.plot(df_sev.index, df_sev['severity_test'], c='darkorange', label='severity_test')
        